In [1]:
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.io as sio
import skimage.io
import h5py

import sys
sys.path.append('../Metric/')
from Metric import *
from Data_Extractor import *

Load & Reorder:

In [2]:
# Load
path_raw_image = "../../Data/090085/090085_20170531.h5"
path_road_mask = "../../Data/090085/Road_Data/motor_trunk_pri_sec_tert/motor_trunk_pri_sec_tert.tif"
path_topleft_coordinate = "../../Data/090085/Road_Data/motor_trunk_pri_sec_tert/topleft_coordinate.h5"

raw_image = np.array(h5py.File(path_raw_image)['scene'])
road_mask = skimage.io.imread(path_road_mask)

data = h5py.File(path_topleft_coordinate, 'r')

pos_topleft_coord = np.array(data['positive_example'])
neg_topleft_coord = np.array(data['negative_example'])
data.close()

print(raw_image.shape, road_mask.shape)
print(raw_image.min())

(7, 7961, 8091) (7961, 8091)
-9999


In [3]:
# Construct training & test set
pos_index_mask = np.arange(pos_topleft_coord.shape[0])
neg_index_mask = np.arange(neg_topleft_coord.shape[0])

np.random.shuffle(pos_index_mask)
np.random.shuffle(neg_index_mask)

# 0 - neg; 1 - pos
train_index = [neg_index_mask[:int(neg_index_mask.size*0.75) ], pos_index_mask[:int(pos_index_mask.size*0.75) ]]
test_index  = [neg_index_mask[ int(neg_index_mask.size*0.75):], pos_index_mask[ int(pos_index_mask.size*0.75):]]

Train_Data = Data_Extractor (raw_image, road_mask, 28, 
                             pos_topleft_coord = pos_topleft_coord[train_index[1]], 
                             neg_topleft_coord = neg_topleft_coord[train_index[0]])

Test_Data  = Data_Extractor (raw_image, road_mask, 28, 
                             pos_topleft_coord = pos_topleft_coord[test_index[1]], 
                             neg_topleft_coord = neg_topleft_coord[test_index[0]])

Model parameters: 

In [4]:
width = 28
height = 28
band = 7

# Hyper parameters
conv_out = [0, 64, 128, 192]
last_conv_flatten = 4*4*conv_out[-1]
layer_out = [0, 256, 128]

class_output = 1 # number of possible classifications for the problem

uni = True # Xavier (uniform / normal)

keep_rate = 0.5 # need regularization => otherwise NaN appears inside CNN

batch_size = 64
learning_rate = 9e-6
iteration = 50000

Calculate Normalization Parameters:

In [5]:
# Normalize Parameters
mu = 0
sigma = 0
step = width
for img in Train_Data:
    mu += img
mu = mu / Train_Data.size

for img in Train_Data:
    sigma += (img-mu)**2
sigma = sigma / Train_Data.size

print(mu.shape,    mu.dtype)
print(sigma.shape, sigma.dtype)

(5488,) float64
(5488,) float64


Place holders for inputs and outputs: 

In [6]:
x = tf.placeholder(tf.float32, shape=[None, height, width, band], name='x')
y = tf.placeholder(tf.float32, shape=[None, class_output], name='y')

keep_prob = tf.placeholder(tf.float32) # dropout
is_training = tf.placeholder(tf.bool, name='phase') # batch norm

Convolutional Layers:

In [7]:
# Convolutional Layer 1
W_conv1 = tf.get_variable('W_conv1', shape = [3, 3, band, conv_out[1]], 
                          initializer=tf.contrib.layers.xavier_initializer(uniform=uni))
b_conv1 = tf.Variable(tf.zeros([conv_out[1]]))

convolve1= tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
bn_1 = tf.contrib.layers.batch_norm(convolve1, center=True, scale=True, is_training=is_training)

h_conv1 = tf.nn.relu(convolve1)
conv1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Convolutional Layer 2
W_conv2 = tf.get_variable("W_conv2", shape = [3, 3, conv_out[1], conv_out[2]], 
                          initializer=tf.contrib.layers.xavier_initializer(uniform=uni))
b_conv2 = tf.Variable(tf.zeros([conv_out[2]]))

convolve2= tf.nn.conv2d(conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')+ b_conv2
bn_2 = tf.contrib.layers.batch_norm(convolve2, center=True, scale=True, is_training=is_training)

h_conv2 = tf.nn.relu(convolve2)
conv2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Convolutional Layer 3
W_conv3 = tf.get_variable("W_conv3", shape = [3, 3, conv_out[2], conv_out[3]], 
                          initializer=tf.contrib.layers.xavier_initializer(uniform=uni))
b_conv3 = tf.Variable(tf.zeros([conv_out[3]]))

convolve3= tf.nn.conv2d(conv2, W_conv3, strides = [1, 1, 1, 1], padding='SAME')+ b_conv3
bn_3 = tf.contrib.layers.batch_norm(convolve3, center=True, scale=True, is_training=is_training)

h_conv3 = tf.nn.relu(convolve3)
conv3 = tf.nn.max_pool(h_conv3, ksize=[1,2,2,1], strides=[1, 2, 2, 1], padding='SAME')

# Flattening
layer2_matrix = tf.reshape(conv3, [-1,last_conv_flatten])

Fully Connected Layer: 

In [8]:
# Layer 1
W_fc1 = tf.get_variable('W_fc1', shape = [last_conv_flatten, layer_out[1]], 
                        initializer=tf.contrib.layers.xavier_initializer(uniform=uni))
b_fc1 = tf.Variable(tf.zeros([layer_out[1]]))

fcl=tf.matmul(layer2_matrix, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(fcl)

# Drop out layer:
layer_drop = tf.nn.dropout(h_fc1, keep_prob)

# Layer 2
W_fc2 = tf.get_variable('W_fc2', shape = [layer_out[1], layer_out[2]], 
                        initializer=tf.contrib.layers.xavier_initializer(uniform=uni))
b_fc2 = tf.Variable(tf.zeros([layer_out[2]]))

fc2=tf.matmul(layer_drop, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(fc2) # ReLU activation

Output Layer (Sigmoid):

In [9]:
W_fc_out = tf.get_variable('W_fc_out', shape = [layer_out[2], class_output], 
                        initializer=tf.contrib.layers.xavier_initializer(uniform=uni))

b_fc_out = tf.Variable(tf.zeros([class_output]))

fc_out = tf.matmul(h_fc2, W_fc_out) + b_fc_out

y_CNN = tf.sigmoid(fc_out, name='y_CNN')
prediction = tf.cast(tf.round(y_CNN), tf.int32, name='prediction')

Cost function & optimizer:

In [10]:
cross_entropy = -tf.reduce_mean(tf.reduce_sum((y * tf.log(y_CNN) + (1-y) * tf.log(1-y_CNN)), axis=1))

# Ensures that we execute the update_ops before performing the train_step
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [11]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(y, tf.round(y_CNN)), "float"))

Train & monitor:

In [ ]:
saver = tf.train.Saver()

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

learning_curve = []
for i in range(iteration):

    # force batch contains at least 1 positive example
    batch_x, batch_y = Train_Data.get_patches(batch_size=64, positive_num=8)
    batch = [((batch_x-mu)/sigma).reshape((-1,band,height,width)).transpose((0,2,3,1)).astype(np.float32), 
             np.matrix(batch_y).astype(int).T]

    train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: keep_rate, is_training: True})
        
    # snap shot
    if i%1000 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y: batch[1], keep_prob: 1})
        learning_curve.append(train_accuracy)
        
        print("acc = ", train_accuracy, " cross entropy = ", 
              cross_entropy.eval(feed_dict={x:batch[0], y: batch[1], keep_prob: 1}))

print("finish")

acc =  0.875  cross entropy =  0.693084


In [ ]:
print(y_CNN.eval(feed_dict={x:batch[0], keep_prob: 1}).T)
print(fc_out.eval(feed_dict={x:batch[0], keep_prob: 0.5}).T)
print(layer2_matrix.eval(feed_dict={x:batch[0]}).min())
print("===")
print(conv3.eval(feed_dict={x:batch[0], keep_prob: 0.5}).flatten())
print(conv2.eval(feed_dict={x:batch[0], keep_prob: 0.5}).flatten())
print(conv1.eval(feed_dict={x:batch[0], keep_prob: 0.5}).flatten())
print(convolve1.eval(feed_dict={x:batch[0], keep_prob: 0.5}).flatten())
print(W_conv1.eval().flatten())
print('=====')
print(batch[0].mean())

In [ ]:
# plot training curve
plt.figsize=(9,5)
plt.plot(learning_curve)
plt.title('learning_curve')
plt.show()

## (Optional) Save the model

In [ ]:
save_path = './Result/motor_trunk_pri_sec_tert/'
model_name = 'motor_trunk_pri_sec_tert'
saver.save(sess, save_path + model_name)

h5f = h5py.File(save_path + "training_info.h5", 'w')

h5f_Index = h5f.create_group("Index")
h5f_Index.create_dataset(name='train_index', shape=train_index.shape, data=train_index)
h5f_Index.create_dataset(name='test_index', shape=test_index.shape, data=test_index)

h5f_Norm = h5f.create_group("Norm")
h5f_Norm.create_dataset(name='mu', shape=mu.shape, data=mu)
h5f_Norm.create_dataset(name='sigma', shape=sigma.shape, data=sigma)
h5f.close()

Evaluate:

In [ ]:
train_metric = Metric()

batch_num = int(train_index.size/batch_size)+1
for i in range(batch_num):
    start = i%batch_num * batch_size
    end = start + batch_size

    if end > train_index.size:
        end = train_index.size
    
    index = train_index[start:end]
    batch_x, batch_y = get_patches(raw_image, road_mask, topleft_coordinate[index], step=width)
    batch = [((batch_x-mu)/sigma).reshape((-1,band,height,width)).transpose((0,2,3,1)).astype(np.float32), 
             np.matrix(batch_y).astype(int).T]
    
    # record metric
    pred = prediction.eval(feed_dict={x:batch[0], keep_prob: 1, is_training: False})
    train_metric.accumulate(pred, batch[1])
    
train_metric.print_info()

In [ ]:
test_metric = Metric()

batch_num = int(test_index.size/batch_size)+1
for i in range(batch_num):
    start = i%batch_num * batch_size
    end = start + batch_size

    if end > test_index.size:
        end = test_index.size
    
    index = test_index[start:end] 
    batch_x, batch_y = get_patches(raw_image, road_mask, topleft_coordinate[index], step=width)
    batch = [((batch_x-mu)/sigma).reshape((-1,band,height,width)).transpose((0,2,3,1)).astype(np.float32), 
             np.matrix(batch_y).astype(int).T]
    # record metric
    pred = prediction.eval(feed_dict={x:batch[0], keep_prob: 1, is_training: False})
    test_metric.accumulate(pred, batch[1])
    
test_metric.print_info()